# Imports

In [ ]:
import packages

from context.utils import typer as t
from context.utils.handlers import print_hldr
from toolkit.utils import utils
from toolkit.utils.utils import rp_print

from context.infra import clients
import context.instances as inst
import context.consts as const
import context.settings.main as settings_main

from toolkit.llm.langchain.core import integration, utils as utils_lc
from toolkit.llm.langchain.data.persistence import retrievers
from toolkit.llm.langchain.data.indexing import (
  documents, document_loaders, text_splitters,
)
from toolkit.llm.langchain.execution import (
  runnables, graphs, tools as tools_lc, agents
)
from toolkit.llm.langchain.models import (
	prompts as prompts_lc, llms, messages as msgs_lc
)

In [ ]:
fw = settings_main.CHOSEN["framework_llm"]

# vector_store = inst.vector_store_in_memory
vector_stores = inst.vector_stores_qdrant

COLLS = settings_main.VEC_STR_COLLS

llm = inst.llm_main
embedding = inst.embedding_main

prompts = prompts_lc.prompts
# prompt_system_rag = prompt_system_rag.replace("{context}", docs_content)


# Intro

In [ ]:
class NODE(t.EnumCustom):
	TOOLS = t.auto()
	AGENT = t.auto()

@tools_lc.tool
def search(query: str):
	"""Call to surf the web."""
	if "sf" in query.lower() or "san francisco" in query.lower():
			return "It's 60 degrees and foggy."
	return "It's 90 degrees and sunny."

tools = [search] 
node_tool = graphs.ToolNode(tools)

llm = llms.create_tooled_llm(llm, tools)

def should_continue(
	state: graphs.MessagesState
) -> t.Literal[NODE.TOOLS, graphs.END]:
	msgs = state["messages"]
	last_msg = msgs[-1]
	# If the LLM makes a tool call, then we route to the "tools" node
	if last_msg.tool_calls:
		return NODE.TOOLS
	# Otherwise, we stop (reply to the user)
	return graphs.END

def agent(state: graphs.MessagesState):
	msgs = state["messages"]
	response = llm.invoke(msgs)
	# We return a list, because this will get added to the existing list
	return {"messages": [response]}

builder = graphs.StateGraph(state_schema=graphs.MessagesState)

builder.add_node(NODE.AGENT, agent)
builder.add_node(NODE.TOOLS, node_tool)

# Set the entrypoint as `agent`; this node is called first.
builder.add_edge(graphs.START, NODE.AGENT)

builder.add_conditional_edges(
	# Define the start node as `agent`; these are the edges taken after `agent` is called.
	NODE.AGENT,
	# function that will determine which node is called next
	should_continue,
)

# Add a normal edge from `tools` to `agent`, so `agent` is called after `tools`.
builder.add_edge(NODE.TOOLS, NODE.AGENT)

checkpointer = graphs.MemorySaver()

# Compile it into a LangChain Runnable, which can be used like any other runnable. 
# Optionally, pass the memory when compiling the graph.
app = builder.compile(checkpointer=checkpointer)

In [ ]:
class NODE(t.EnumCustom):
	# Node can be either an LLM or an agent.
	BOT = t.auto()
	TOOLS = t.auto()
	HUMAN = t.auto()

	MANAGER = t.auto() 
	LEAD_ = t.auto()
	MEMBER_ = t.auto()

	AGENT_ = t.auto() # An agent can also be defined as a tool.

class FEATURE:
  ASK_HUMAN = False

class ARCHITECTURE(t.EnumCustom):
  NETWORK = t.auto()

# Conceptual

# Tutorials

## Quick Start


### 🚀 LangGraph Quick Start

In [ ]:
class FEATURE:
  ASK_HUMAN = False

class NODE(t.EnumCustom):
	BOT = t.auto()
	TOOLS = t.auto()
	HUMAN = t.auto()

class State(t.TypedDict):
	# Messages are of type "list".
	# The `add_messages` function in the annotation defines how this state key 
	# should be updated, appending messages to the list rather than overwriting them.
	messages: t.Annotated[list, graphs.add_messages]

	if FEATURE.ASK_HUMAN:
		ask_human: bool

class ToolRequestAssistance(t.BaseModel):
	"""
	Escalate the conversation to an expert. Use this if you are unable to assist directly or if the user requires support beyond your permissions.
	To use this function, relay the user's 'request' so the expert can provide the right guidance.
 	"""
	request: str

tools = [
	tools_lc.tool_search_tavily,
]

if FEATURE.ASK_HUMAN: tools.append(ToolRequestAssistance)

node_tool = graphs.ToolNode(tools)

llm = llms.create_tooled_llm(llm, tools)

def node_bot(state: State) -> State:
	response: msgs_lc.AIMessage = llm.invoke(state["messages"])
 
	result = {
		"messages": [
			response
		],
	}	
 
	if FEATURE.ASK_HUMAN:
		ask_human = False
	
		if (
			response.tool_calls and 
			response.tool_calls[0]["name"] == ToolRequestAssistance.__name__
		):
			ask_human = True,
   
		result["ask_human"] = ask_human
 
	return result

def node_human(state: State) -> State:
	messages_new = []

	if not isinstance(state["messages"][-1], msgs_lc.ToolMessage):
		# Typically, the user will have updated the state during the interrupt.
		# If they choose not to, we will include a placeholder ToolMessage to
		# let the LLM continue.
		
		response = input("Please provide the expert guidance.")
  
		messages_new.append(msgs_lc.respond_tool_calling(
			response=response,
			message_ai=state["messages"][-1],
		))
	
	return {
		"messages": messages_new,
		"ask_human": False,
	}

def router(state: State):
	if FEATURE.ASK_HUMAN:
		if state["ask_human"]:
			return NODE.HUMAN
	return graphs.tools_condition(state)

builder = graphs.StateGraph(State)

# The first argument is the unique node name, and the second argument is the 
# function or object called whenever the node is used.
builder.add_node(NODE.BOT, node_bot)
builder.add_node(NODE.TOOLS, node_tool)
builder.add_node(NODE.HUMAN, node_human)

builder.add_edge(graphs.START, NODE.BOT)

builder.add_conditional_edges(
  NODE.BOT, 
  router,
	{
		NODE.HUMAN: NODE.HUMAN,
		NODE.TOOLS: NODE.TOOLS,
		graphs.END: graphs.END,
	}
)

builder.add_edge(NODE.TOOLS, NODE.BOT)
builder.add_edge(NODE.HUMAN, NODE.BOT)

memory = graphs.MemorySaver()

app = builder.compile(
  checkpointer=memory,
	interrupt_before=[
		NODE.TOOLS,
		NODE.HUMAN,
  ],
	# interrupt_after=[NODE.TOOLS],
)

# graphs.display(app)

In [ ]:
inputs = [
	"Hello",
	"Hi there! My name is Will.",
	"Remember my name?",
	"What do you know about LangGraph?",
	"I'm learning LangGraph. Could you do some research on it for me?",
	"I need some expert guidance for building this AI agent. Could you request assistance for me?",

]
user_input = inputs[5]

config = {"configurable": {"thread_id": "1"}}

# for event in app.stream(
#   input={"messages": [("user", user_input)]},
# 	config=config, stream_mode="values"
# ):
# 	event["messages"][-1].pretty_print()

while True:
	for event in app.stream(
		input={"messages": [("user", user_input)]},
		config=config, stream_mode="values"
	):
		event["messages"][-1].pretty_print()
	
	snapshot = graphs.get_snapshot(app, config)
	message_current: t.Union[msgs_lc.BaseMessage, msgs_lc.AIMessage] = snapshot.values["messages"][-1]

	is_tool_calling = msgs_lc.is_tool_calling(message_current)
	
	if is_tool_calling:
		tool_name = message_current.tool_calls[0]['name']
		
		if tool_name == tools_lc.tool_search_tavily.name:
			tool_query = message_current.tool_calls[0]['args']["query"]

			if input(f"Would you like me to begin with this input? `{tool_query}").lower() in ["no"]:
				input_new = input("Please enter your desired input.")
				app = graphs.update_last_tool_calling(app, config, input_new=input_new)

			if input("Would you like to continue?").lower() in ["yes", "ok"]:
				for event in app.stream(
					input=None,
					config=config, stream_mode="values"
				):
					event["messages"][-1].pretty_print()
			else:
				break
		
		if tool_name == ToolRequestAssistance.__name__:
			response_human = (
				"We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
				" It's much more reliable and extensible than simple autonomous agents."
			)
			message_tool = msgs_lc.respond_tool_calling(response_human, message_current)
			
			app.update_state(config, {"messages": [message_tool]})

			for event in app.stream(
				input=None,
				config=config, stream_mode="values"
			):
				event["messages"][-1].pretty_print()
	
	break

## Chatbots


## RAG


## Agent Architectures


### Multi-Agent Systems


#### Network


In [ ]:
class NODE(t.EnumCustom):
	AGENT_RESEARCH = t.auto()
	AGENT_CHART = t.auto()

def get_next_node(message_current: msgs_lc.BaseMessage, goto: str):
	if "FINAL ANSWER" in message_current.content:
		return graphs.END
	return goto

agent_research = agents.create_react_agent(
	model=llm,
	tools=[tools_lc.tool_search_tavily],
	state_modifier=prompts["agents"]["tpl_system_network"].replace(
		"{suffix}",
		"You can only do research. You are working with a chart generator colleague.",
	)
)

def node_agent_research(
	state: graphs.MessagesState,
) -> graphs.Command[t.Literal[NODE.AGENT_CHART, graphs.END]]:
	rp_print(state)
	result = agent_research.invoke(state)
	rp_print(result)
	message_current = graphs.get_lastest_message(result)
	
	goto = get_next_node(message_current, NODE.AGENT_CHART)

	message_current = graphs.manipulate_message(
		message=message_current, type_msg=msgs_lc.TypeMsg.HUMAN,
		name=NODE.AGENT_RESEARCH,
	)
 
	# wrap in a human message, as not all providers allow
	# AI message at the last position of the input messages list
	result["messages"][-1] = message_current
	
	return graphs.Command(
		update={
			# share internal message history of between agents
			"messages": result["messages"],
		},
		goto=goto,
	)

agent_chart = agents.create_react_agent(
	model=llm,
	tools=[tools_lc.tool_python_repl],
	state_modifier=prompts["agents"]["tpl_system_network"].replace(
		"{suffix}",
		"You can only generate charts. You are working with a researcher colleague.",
	)
)

def node_agent_chart(
	state: graphs.MessagesState,
) -> graphs.Command[t.Literal[NODE.AGENT_RESEARCH, graphs.END]]:
	result = agent_chart.invoke(state)
	message_current = graphs.get_lastest_message(result)
	
	goto = get_next_node(message_current, NODE.AGENT_RESEARCH)

	message_current = graphs.manipulate_message(
		message=message_current, type_msg=msgs_lc.TypeMsg.HUMAN,
		name=NODE.AGENT_CHART,
	)
 
	# wrap in a human message, as not all providers allow
	# AI message at the last position of the input messages list
	result["messages"][-1] = message_current
 
	return graphs.Command(
		update={
			# share internal message history of between agents
			"messages": result["messages"],
		},
		goto=goto,
	)

builder = graphs.StateGraph(graphs.MessagesState)

builder.add_node(NODE.AGENT_RESEARCH, node_agent_research)
builder.add_node(NODE.AGENT_CHART, node_agent_chart)

builder.add_edge(graphs.START, NODE.AGENT_RESEARCH)

app = builder.compile()

# graphs.display_graph(app)

In [ ]:
user_input = (
	"First, get the UK's GDP over the past 5 years, then make a line chart of it. "
	"Once you make the chart, finish.",
)

for event in app.stream(
	input={"messages": [("user", user_input)]},
	# config=config, stream_mode="values"
):
	# event["messages"][-1].pretty_print()
	rp_print(event)

#### Supervisor


#### Hierarchical Teams


### Planning Agents


#### Plan-and-Execute


#### Reasoning without Observation


#### LLMCompiler


### Reflection & Critique


#### Basic Reflection


#### Reflexion


#### Tree of Thoughts


#### Language Agent Tree Search


#### Self-Discover Agent


## Evaluation & Analysis


## Experimental


## LangGraph Platform

# How-to

## LangGraph


### Controllability


#### How to create branches for parallel execution


#### How to create map-reduce branches for parallel execution


#### How to control graph recursion limit


#### How to combine control flow and state updates with Command


### Persistence


#### How to add thread-level persistence to your graph


#### How to add thread-level persistence to subgraphs


#### How to add cross-thread persistence to your graph


#### How to use Postgres checkpointer for persistence


#### How to use MongoDB checkpointer for persistence


#### How to create a custom checkpointer using Redis


### Memory


#### How to manage conversation history


#### How to delete messages


#### How to add summary conversation memory


#### How to add long-term memory (cross-thread)


#### How to use semantic search for long-term memory


### Human-in-the-loop


#### How to wait for user input


#### How to review tool calls


#### How to add static breakpoints


#### How to edit graph state


#### How to add dynamic breakpoints with NodeInterrupt


### Time Travel


#### How to view and update past graph state


### Streaming


#### How to stream
#### How to stream LLM tokens from specific nodes
#### How to stream data from within a tool



In [ ]:
class State(t.TypedDict):
    topic: str
    joke: str
    poem: str

async def node_generate(state: State, config: runnables.RunnableConfig) -> State:
    topic = state["topic"]

    print_hldr("Writing joke ...")
    response_joke = await inst.llm_main.ainvoke(
        [msgs_lc.HumanMessage(f"Write a joke about {topic}")],
        config=config,
    )

    print_hldr("\n\nWriting poem ...")
    response_poem = await inst.llm_main.ainvoke(
        [msgs_lc.HumanMessage(f"Write a short poem about {topic}")],
        config=config,
    )

    return {
        'joke': response_joke.content,
        'poem': response_poem.content,
    }


@tools_lc.tool
async def get_something(
    the_input: t.Any, config: runnables.RunnableConfig
) -> t.Any:
    """
    Use this tool to get something from the input.
    """

    result: msgs_lc.AnyMessage = await clients.error_handler_silent.execute(
        lambda: inst.llm_main.ainvoke(
            [msgs_lc.HumanMessage(f"Get something from {the_input}")],
            config=config,
        )
    )

    result = clients.error_handler_silent.execute(lambda: result.content)

    return result

builder = graphs.StateGraph(State)

builder.add_node(node_generate)

builder.add_edge(graphs.START, node_generate.__name__)
builder.add_edge(node_generate.__name__, graphs.END)

graph = builder.compile()

In [ ]:
async for msg_chunk, metadata in graph.astream(
    {
        "topic": "ice cream",
    },
    stream_mode="messages",
):  
    msg_chunk: msgs_lc.AnyMessage
    metadata: t.Dict
    
    cond_state = "joke" in metadata.get("tags", [])
    cond_node = metadata["langgraph_node"] == node_generate.__name__
    cond_tool = metadata["langgraph_node"] == "tools"

    if msg_chunk.content and cond_node:
        print_hldr(msg_chunk.content, end="", flush=True)

In [ ]:
for chunk in graph.stream(
    {
        "topic": "ice cream",
    },
    stream_mode="values",
):
    print(chunk)

In [ ]:
async for event in graph.astream(
	input={"topic": "ice cream"},
	stream_mode="values"
):
	print(event)

#### How to stream from subgraphs


In [ ]:
class StateSub(t.TypedDict):
    foo: str # note that this key is shared with the parent graph state
    bar: str

def node_sub_1(state: StateSub) -> StateSub:
    return {
        "bar": "bar",
    }

def node_sub_2(state: StateSub) -> StateSub:
    return {
        "foo": state["foo"] + state["bar"]
    }

builder_sub = graphs.StateGraph(StateSub)
builder_sub.add_node(node_sub_1)
builder_sub.add_node(node_sub_2)
builder_sub.add_edge(graphs.START, node_sub_1.__name__)
builder_sub.add_edge(node_sub_1.__name__, node_sub_2.__name__)
builder_sub.add_edge(node_sub_2.__name__, graphs.END)
graph_sub = builder_sub.compile()

In [ ]:
class StateMain(t.TypedDict):
    foo: str # note that this key is shared with the child graph state

def node_main_1(state: StateMain) -> StateMain:
    return {
        "foo": "hi!" + state["foo"]
    }

builder_main = graphs.StateGraph(StateMain)
builder_main.add_node("node_1", node_main_1)
builder_main.add_node("node_2", graph_sub)
builder_main.add_edge(graphs.START, "node_1")
builder_main.add_edge("node_1", "node_2")
builder_main.add_edge("node_2", graphs.END)
graph_main = builder_main.compile()

In [ ]:
for chunk in graph_main.stream(
    {"foo": "foo"},
    stream_mode="updates",
    subgraphs=True,
):
    print(chunk)
    pass

#### How to disable streaming for models that don't support it

### Tool calling


#### How to call tools using ToolNode


#### How to handle tool calling errors


#### How to pass runtime values to tools


#### How to pass config to tools


#### How to update graph state from tools


#### How to handle large numbers of tools


### Subgraphs


#### How to use subgraphs


In [ ]:
# Subgraph nodes
class NODES_SUB(t.EnumCustom):
    STEP1 = t.auto()    
    STEP2 = t.auto()    

# Subgraph state
class State_Sub(t.TypedDict):
    input_text: str    
    helper_text: str   

def node_sub_step1(state: State_Sub) -> dict:
    """First subgraph step: adds helper text"""
    return {
        "helper_text": "HELPER"
    }

def node_sub_step2(state: State_Sub) -> dict:
    """Second subgraph step: combines input with helper"""
    return {
        "input_text": f"{state['input_text']}_{state['helper_text']}"
    }

# Build subgraph
builder_sub = graphs.StateGraph(State_Sub)
builder_sub.add_node(NODES_SUB.STEP1, node_sub_step1)
builder_sub.add_node(NODES_SUB.STEP2, node_sub_step2)
builder_sub.add_edge(graphs.START, NODES_SUB.STEP1)
builder_sub.add_edge(NODES_SUB.STEP1, NODES_SUB.STEP2)
graph_sub = builder_sub.compile()

# Main graph nodes
class NODES_MAIN(t.EnumCustom):
    STEP1 = t.auto()    
    STEP2 = t.auto()    

# Main graph state
class State_Main(t.TypedDict):
    text: str    # Text being processed

def node_main_step1(state: State_Main) -> dict:
    """First main step: formats text"""
    return {
        "text": f"Main_{state['text']}"
    }

def node_main_step2(state: State_Main) -> dict:
    """Second main step: uses subgraph to enhance text"""
    # Call subgraph with current text
    sub_result = graph_sub.invoke({"input_text": state["text"]})
    
    return {
        "text": sub_result["input_text"]
    }

# Build main graph
builder_main = graphs.StateGraph(State_Main)
builder_main.add_node(NODES_MAIN.STEP1, node_main_step1)
builder_main.add_node(NODES_MAIN.STEP2, node_main_step2)
builder_main.add_edge(graphs.START, NODES_MAIN.STEP1)
builder_main.add_edge(NODES_MAIN.STEP1, NODES_MAIN.STEP2)
graph_main = builder_main.compile()

# Test the graph
for chunk in graph_main.stream({"text": "Test"}, subgraphs=True):
    print(chunk)

#### How to view and update state in subgraphs


#### How to transform inputs and outputs of a subgraph


### Multi-agent


#### How to implement handoffs between agents


In [ ]:
from typing import Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.types import Command

model = inst.llm_main

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

@tool
def transfer_to_multiplication_expert():
    """Transfer the calculation to the multiplication expert."""
    return

@tool
def transfer_to_addition_expert():
    """Transfer the calculation to the addition expert."""
    return

def has_calculations_remaining(messages: list[BaseMessage]) -> bool:
    if not messages:
        return True
    last_msg = messages[-1]
    content = last_msg.content if isinstance(last_msg, AIMessage) else str(last_msg)
    return not (any(phrase in content.lower() for phrase in ["final result is", "the result is", "= ", "equals"]) 
                and any(char.isdigit() for char in content))

def process_tools(response: AIMessage, messages: list) -> tuple[list, bool]:
    updated_msgs = messages + [response]
    transfer = False
    
    if hasattr(response, 'tool_calls') and response.tool_calls:
        for tool_call in response.tool_calls:
            name, call_id = tool_call["name"], tool_call["id"]
            if name in ["add", "multiply"]:
                result = globals()[name].invoke(tool_call)
                updated_msgs.append(ToolMessage(content=str(result), tool_call_id=call_id))
            elif "transfer" in name:
                updated_msgs.append(ToolMessage(content="Transfer request acknowledged", tool_call_id=call_id))
                transfer = True
    
    return updated_msgs, transfer

def create_expert(expert_type: str):
    def expert(state: MessagesState) -> Command[Literal["multiplication_expert", "addition_expert", "__end__"]]:
        messages = state["messages"]
        if not has_calculations_remaining(messages):
            return Command(goto="__end__", update={"messages": messages})

        system_msg = SystemMessage(content=(
            f"You are a {expert_type} expert. Follow these rules:\n"
            f"1. If you see a {expert_type} problem, solve it using the {expert_type.split()[0]} tool\n"
            f"2. If you see a {'multiplication' if 'addition' in expert_type else 'addition'} problem, "
            f"use transfer_to_{'multiplication' if 'addition' in expert_type else 'addition'}_expert\n"
            "3. Once you've done the calculation, state the result clearly\n"
        ))

        tools = [add if "addition" in expert_type else multiply,
                transfer_to_multiplication_expert if "addition" in expert_type else transfer_to_addition_expert]
        response = model.bind_tools(tools).invoke([system_msg] + messages)
        updated_msgs, should_transfer = process_tools(response, messages)
        
        return Command(
            goto="multiplication_expert" if "addition" in expert_type else "addition_expert" if should_transfer else "__end__",
            update={"messages": updated_msgs}
        )
    return expert

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("addition_expert", create_expert("addition"))
builder.add_node("multiplication_expert", create_expert("multiplication"))
builder.add_edge(START, "addition_expert")
graph = builder.compile()

def run_calculation(question: str):
    """Run a calculation through the expert system."""
    for chunk in graph.stream({"messages": [HumanMessage(content=question)]}):
        if isinstance(chunk, tuple):
            ns, update = chunk
            if not ns:
                continue
            print(f"Update from subgraph {ns[-1].split(':')[0]}:\n")
        else:
            for node_name, node_update in chunk.items():
                print(f"Update from node {node_name}:\n")
                for m in node_update["messages"]:
                    if isinstance(m, (HumanMessage, AIMessage, SystemMessage, ToolMessage)):
                        m.pretty_print()
                    else:
                        print(m)
                print("\n")

# Example usage
run_calculation("what's (3 + 5) * 12")

In [ ]:
import inspect
from typing import Any, Optional
from pydantic import create_model, BaseModel

def create_transfer_tool(description: str, name: str = None):
    # Get the calling frame
    frame = inspect.currentframe().f_back
    
    # If name not provided, try to get the variable name from assignment
    if name is None:
        # Get the code context from the frame
        context = inspect.getframeinfo(frame).code_context
        if context:
            # Find the variable name from assignment
            caller_lines = "".join(context)
            assignment = caller_lines.split("=")[0].strip()
            name = assignment
    
    # Fallback if we couldn't determine the name
    if not name:
        name = "transfer_tool"
        
    # Create a simple schema with message field
    schema = create_model(
        f"{name}_schema",
        __base__=BaseModel,
        query=(str, None)  # Optional message field for any additional info
    )
    
    return tools_lc.StructuredTool(
        name=name,
        description=description,
        func=lambda query=None: None,  # Simple pass-through function
        args_schema=schema
    )

# Usage example - much simpler now
tool_transfer_to_agent_mul = create_transfer_tool(
    description="Ask multiplication agent for help."
)

tool_transfer_to_agent_add = create_transfer_tool(
    description="Ask addition agent for help."
)

# llm = llms.create_tooled_llm(
#     inst.llm_main,
#     tools=[tool_transfer_to_agent_mul, tool_transfer_to_agent_add]
# )
# result = llm.invoke("How to add one and two multiply three")
# rp_print(result)


In [ ]:
# Define registry with proper framework typing
TOOL_REGISTRY: t.Dict[str, tools_lc.BaseTool] = {}

def process_tools(response: msgs_lc.AIMessage, messages: list) -> tuple[list, bool]:
    """
    Process tool calls from the LLM response.
    
    Args:
        response: The AI message containing tool calls
        messages: Current message history
        
    Returns:
        tuple[list, bool]: Updated messages and transfer flag
    """
    updated_msgs = messages + [response]
    transfer = False
    
    if hasattr(response, 'tool_calls') and response.tool_calls:
        for tool_call in response.tool_calls:
            name = tool_call["name"]
            call_id = tool_call["id"]
            
            if name in TOOL_REGISTRY:
                # Use the tool's built-in invoke method
                tool = TOOL_REGISTRY[name]
                result = tool.invoke(tool_call)
                
                # Extract just the value from the result
                if isinstance(result, str) and 'content=' in result:
                    content = result.split("'")[1]
                else:
                    content = str(result)
                
                tool_msg = msgs_lc.ToolMessage(
                    content=content,
                    tool_call_id=call_id
                )
                updated_msgs.append(tool_msg)
            elif "tool_transfer_to_agent" in name:
                tool_msg = msgs_lc.ToolMessage(
                    content="Transfer request acknowledged",
                    tool_call_id=call_id
                )
                updated_msgs.append(tool_msg)
                transfer = True
    
    return updated_msgs, transfer

@tools_lc.tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

@tools_lc.tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

# Register tools after creation
TOOL_REGISTRY['add'] = add
TOOL_REGISTRY['multiply'] = multiply

In [ ]:
class NODE(t.EnumCustom):
	AGENT_ADD = t.auto()
	AGENT_MUL = t.auto()
	
# Define template for system prompts
SYSTEM_PROMPT_TEMPLATE = """You are an expert agent specializing in {domain}. 

ROLE AND RESPONSIBILITIES:
1. You are responsible for {primary_task}
2. Stay focused on your specific expertise area
3. Collaborate with other agents when needed

TOOL USAGE PRINCIPLES:
1. Use your assigned tools effectively and appropriately
2. Make all necessary tool calls in a single response
3. Validate inputs before using tools
4. Process results clearly and accurately

COLLABORATION RULES:
1. Transfer to other experts when task is outside your expertise
2. Always complete your part before transferring
3. Provide clear context when transferring
4. Never transfer back to an agent that just transferred to you

RESPONSE STRUCTURE:
1. Analyze the task first
2. Use appropriate tools as needed
3. Show work clearly and step by step
4. Transfer only when necessary

{specific_instructions}
"""

# Specific instructions for each agent type
ADD_SPECIFIC = """
KEY RULES FOR ADDITION EXPERT:
1. PATTERN RECOGNITION:
	When you see expressions like "(a + b) * c", this ALWAYS requires TWO tool calls:
	- First call: add tool for (a + b)
	- Second call: transfer to multiplication expert for the result * c
	You MUST make BOTH calls in the SAME response.

2. REQUIRED TOOL SEQUENCE:
	For ANY expression involving multiplication after addition:
	Step 1: Use add tool to calculate the addition
	Step 2: IMMEDIATELY use tool_transfer_to_agent_mul in the SAME response
	DO NOT wait for another interaction to transfer.

3. EXAMPLE SEQUENCES:
	For "(3 + 5) * 12":
	- CORRECT (do this):
		1. add(a=3, b=5)
		2. tool_transfer_to_agent_mul(query="8 * 12")
	- INCORRECT (don't do this):
		× Only calling add without transfer
		× Waiting for next message to transfer

4. MANDATORY ACTIONS:
	- NEVER handle addition alone if multiplication follows
	- ALWAYS make both tool calls in one response
	- ALWAYS transfer after completing addition
"""

MUL_SPECIFIC = """
MULTIPLICATION EXPERTISE:
1. You handle multiplication operations using the 'multiply' tool
2. Transfer to addition expert if addition is needed first
3. Complete multiplications when numbers are ready
4. Present final results clearly

Example workflow:
For received "8 * 12":
1. multiply(a=8, b=12)  # Calculate final result
"""
	
def agent_addition(
		state: graphs.MessagesState,
) -> graphs.Command[t.Literal[NODE.AGENT_MUL, graphs.END]]:
		model = llms.create_tooled_llm(inst.llm_main, [tool_transfer_to_agent_mul, add])
		prompt_system = SYSTEM_PROMPT_TEMPLATE.format(
				domain="mathematical addition",
				primary_task="handling addition operations and coordinating with multiplication expert",
				specific_instructions=ADD_SPECIFIC
		)
		msgs = [msgs_lc.SystemMessage(prompt_system)] + state["messages"]
		msg_ai: msgs_lc.AIMessage = model.invoke(msgs)

		# Process tool calls and get updated messages
		updated_msgs, should_transfer = process_tools(msg_ai, state["messages"])
		
		if should_transfer:
				return graphs.Command(
						goto=NODE.AGENT_MUL,
						update={"messages": updated_msgs}
				)
		
		return {"messages": updated_msgs}

def agent_multiplication(
		state: graphs.MessagesState,
) -> graphs.Command[t.Literal[NODE.AGENT_ADD, graphs.END]]:
		model = llms.create_tooled_llm(inst.llm_main, [tool_transfer_to_agent_add, multiply])
		prompt_system = SYSTEM_PROMPT_TEMPLATE.format(
				domain="mathematical multiplication",
				primary_task="handling multiplication operations and coordinating with addition expert",
				specific_instructions=MUL_SPECIFIC
		)
		msgs = [msgs_lc.SystemMessage(prompt_system)] + state["messages"]
		msg_ai: msgs_lc.AIMessage = model.invoke(msgs)

		# Process tool calls and get updated messages
		updated_msgs, should_transfer = process_tools(msg_ai, state["messages"])
		
		if should_transfer:
				return graphs.Command(
						goto=NODE.AGENT_ADD,
						update={"messages": updated_msgs}
				)
		
		return {"messages": updated_msgs}

builder = graphs.StateGraph(graphs.MessagesState)

builder.add_node(NODE.AGENT_ADD, agent_addition)
builder.add_node(NODE.AGENT_MUL, agent_multiplication)

builder.add_edge(graphs.START, NODE.AGENT_ADD)

graph = builder.compile()


In [ ]:
def pretty_print_stream(graph_stream):
    """
    Pretty prints the stream from a LangGraph, showing only new messages at each turn.
    
    Args:
        graph_stream: Iterator from graph.stream()
    """
    seen_message_ids = set()
    
    def print_new_messages(messages):
        """Helper function to print only unseen messages."""
        for msg in messages:
            # Skip if we've seen this message before
            if hasattr(msg, 'id') and msg.id in seen_message_ids:
                continue
                
            # Add to seen messages if it has an ID
            if hasattr(msg, 'id'):
                seen_message_ids.add(msg.id)
            
            # Print the message content based on type
            if hasattr(msg, 'content') and msg.content:
                print(f"Message: {msg.content}")
            
            # Print tool calls if present
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tool_call in msg.tool_calls:
                    tool_name = tool_call.get('name', 'unknown_tool')
                    tool_args = tool_call.get('args', {})
                    print(f"Tool Call: {tool_name}")
                    print(f"Arguments: {tool_args}")
            
            # Print tool message results
            if hasattr(msg, 'tool_call_id'):
                print(f"Tool Result: {msg.content}")
            
            if hasattr(msg, 'content') or hasattr(msg, 'tool_calls') or hasattr(msg, 'tool_call_id'):
                print("-" * 50)
    
    for chunk in graph_stream:
        if isinstance(chunk, tuple):
            # Handle subgraph updates
            ns, update = chunk
            if not ns:
                continue
            print(f"\n=== Update from subgraph {ns[-1].split(':')[0]} ===")
            if 'messages' in update:
                print_new_messages(update['messages'])
        else:
            # Handle regular node updates
            for node_name, node_update in chunk.items():
                print(f"\n=== Update from {node_name} ===")
                if 'messages' in node_update:
                    print_new_messages(node_update['messages'])

content = "what's (3 + 5) * 12"
# content = "what's 3*3 + 1"

pretty_print_stream(
		graph.stream({"messages": [msgs_lc.HumanMessage(content=content)]})
)

In [ ]:
result = []
for chunk in graph.stream(
		{"messages": [("user", "what's ((3 + 5) * 12) + 123")]},
		# stream_mode="values"
):
		# rp_print(chunk)
		result.append(chunk)

In [ ]:
# rp_print(result[-1].keys())

from langchain_core.messages import convert_to_messages


def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

for chunk in graph.stream(
    {"messages": [("user", "what's (3 + 5) * 12")]},
):
    pretty_print_messages(chunk)

#### How to build a multi-agent network 


In [ ]:
import random
from typing_extensions import Literal
from typing_extensions import Literal

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.types import Command

@tool
def get_travel_recommendations():
    """Get recommendation for travel destinations"""
    return random.choice(["aruba", "turks and caicos"])


@tool
def get_hotel_recommendations(location: Literal["aruba", "turks and caicos"]):
    """Get hotel recommendations for a given destination."""
    return {
        "aruba": [
            "The Ritz-Carlton, Aruba (Palm Beach)"
            "Bucuti & Tara Beach Resort (Eagle Beach)"
        ],
        "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"],
    }[location]
    
from typing import Annotated

from langchain_core.tools import tool
from langchain_core.tools.base import InjectedToolCallId
from langgraph.prebuilt import InjectedState


def make_handoff_tool(*, agent_name: str):
    """Create a tool that can return handoff via a Command"""
    tool_name = f"transfer_to_{agent_name}"

    @tool(tool_name)
    def handoff_to_agent(
        state: Annotated[dict, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        """Ask another agent for help."""
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": tool_name,
            "tool_call_id": tool_call_id,
        }
        return Command(
            # navigate to another agent node in the PARENT graph
            goto=agent_name,
            graph=Command.PARENT,
            # This is the state update that the agent `agent_name` will see when it is invoked.
            # We're passing agent's FULL internal message history AND adding a tool message to make sure
            # the resulting chat history is valid.
            update={"messages": state["messages"] + [tool_message]},
        )

    return handoff_to_agent
  
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command


model = inst.llm_main

# Define travel advisor ReAct agent
travel_advisor_tools = [
    get_travel_recommendations,
    make_handoff_tool(agent_name="hotel_advisor"),
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    state_modifier=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_travel_advisor(
    state: MessagesState,
) -> Command[Literal["hotel_advisor", "__end__"]]:
    # You can also add additional logic like changing the input to the agent / output from the agent, etc.
    # NOTE: we're invoking the ReAct agent with the full history of messages in the state
    return travel_advisor.invoke(state)


# Define hotel advisor ReAct agent
hotel_advisor_tools = [
    get_hotel_recommendations,
    make_handoff_tool(agent_name="travel_advisor"),
]
hotel_advisor = create_react_agent(
    model,
    hotel_advisor_tools,
    state_modifier=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_hotel_advisor(
    state: MessagesState,
) -> Command[Literal["travel_advisor", "__end__"]]:
    return hotel_advisor.invoke(state)


builder = StateGraph(MessagesState)
builder.add_node("travel_advisor", call_travel_advisor)
builder.add_node("hotel_advisor", call_hotel_advisor)
# we'll always start with a general travel advisor
builder.add_edge(START, "travel_advisor")

graph = builder.compile()
# display(Image(graph.get_graph().draw_mermaid_png()))

for chunk in graph.stream(
    {
        "messages": [
            (
                "user",
                "i wanna go somewhere warm in the caribbean. pick one destination and give me hotel recommendations",
            )
        ]
    },
    subgraphs=True,
):
    rp_print(chunk)

In [ ]:
from typing_extensions import Literal

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.types import Command


model = inst.llm_main


# Define a helper for each of the agent nodes to call


@tool
def transfer_to_travel_advisor():
    """Ask travel advisor for help."""
    # This tool is not returning anything: we're just using it
    # as a way for LLM to signal that it needs to hand off to another agent
    # (See the paragraph above)
    return


@tool
def transfer_to_hotel_advisor():
    """Ask hotel advisor for help."""
    return


def travel_advisor(
    state: MessagesState,
) -> Command[Literal["hotel_advisor", "__end__"]]:
    system_prompt = (
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. You must not ask back user, if any information not clear, just suggest and respond."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = model.bind_tools([transfer_to_hotel_advisor]).invoke(messages)
    # If there are tool calls, the LLM needs to hand off to another agent
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        # NOTE: it's important to insert a tool message here because LLM providers are expecting
        # all AI messages to be followed by a corresponding tool result message
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="hotel_advisor", update={"messages": [ai_msg, tool_msg]})

    # If the expert has an answer, return it directly to the user
    return {"messages": [ai_msg]}


def hotel_advisor(
    state: MessagesState,
) -> Command[Literal["travel_advisor", "__end__"]]:
    system_prompt = (
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help. You must not ask back user, if any information not clear, just suggest and respond."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = model.bind_tools([transfer_to_travel_advisor]).invoke(messages)
    # If there are tool calls, the LLM needs to hand off to another agent
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        # NOTE: it's important to insert a tool message here because LLM providers are expecting
        # all AI messages to be followed by a corresponding tool result message
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="travel_advisor", update={"messages": [ai_msg, tool_msg]})

    # If the expert has an answer, return it directly to the user
    return {"messages": [ai_msg]}


builder = StateGraph(MessagesState)
builder.add_node("travel_advisor", travel_advisor)
builder.add_node("hotel_advisor", hotel_advisor)
# we'll always start with a general travel advisor
builder.add_edge(START, "travel_advisor")

graph = builder.compile()

from IPython.display import display, Image

# display(Image(graph.get_graph().draw_mermaid_png()))

from langchain_core.messages import convert_to_messages


def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

for chunk in graph.stream(
    {"messages": [("user", "i wanna go somewhere warm in the caribbean. pick one destination and give me hotel recommendations")]}
):
    pretty_print_messages(chunk)

#### Build

In [ ]:
from typing import Literal, List, Dict, Any, Optional, TypeVar, Generic
from typing_extensions import Literal, TypedDict
from dataclasses import dataclass
from enum import Enum
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from langchain_core.messages import (
    HumanMessage, 
    AIMessage,
    SystemMessage,
    ToolMessage,
    BaseMessage
)
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command

# Initialize rich console
console = Console()

# Use the provided model
model = inst.llm_main

# ============= Core Types =============
class AgentRole(str, Enum):
    WORKER = "worker"
    SUPERVISOR = "supervisor"

@dataclass
class AgentConfig:
    agent_id: str
    role: AgentRole
    system_prompt: str
    allowed_tools: List[Any]
    allowed_handoffs: List[str]

class State(MessagesState):
    """Base state class with context tracking"""
    context: Dict[str, Any]

T = TypeVar('T', bound=State)

# ============= Calculator Tools =============
@tool
def add(a: float, b: float) -> float:
    """Adds two numbers."""
    result = a + b
    console.print(Panel(
        f"add({a}, {b}) = {result}",
        title="[#4B9EF0]Tool Execution[/#4B9EF0]",
        border_style="#4B9EF0"
    ))
    return result

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    result = a * b
    console.print(Panel(
        f"multiply({a}, {b}) = {result}",
        title="[#4B9EF0]Tool Execution[/#4B9EF0]",
        border_style="#4B9EF0"
    ))
    return result

@tool
def transfer_to_multiplication_expert():
    """Transfer to multiplication expert."""
    console.print(Panel(
        "Transferring to multiplication expert",
        title="[#F0A732]Transfer[/#F0A732]",
        border_style="#F0A732"
    ))
    return

@tool
def transfer_to_addition_expert():
    """Transfer to addition expert."""
    console.print(Panel(
        "Transferring to addition expert",
        title="[#F0A732]Transfer[/#F0A732]",
        border_style="#F0A732"
    ))
    return

# ============= Base Classes =============
class BaseAgent:
    """Base class for all agents"""
    def __init__(self, config: AgentConfig):
        self.config = config

    def process_tools(self, response: AIMessage, state: State) -> tuple[State, Optional[str]]:
        """Process tool usage and determine next agent"""
        updated_state = state.copy()
        updated_state["messages"] = state["messages"] + [response]
        next_agent = None

        if hasattr(response, 'tool_calls') and response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                
                # Handle transfers
                if tool_name.startswith("transfer_to_"):
                    target_agent = tool_name.replace("transfer_to_", "")
                    if target_agent in self.config.allowed_handoffs:
                        next_agent = target_agent
                        return updated_state, next_agent

                # Handle regular tools
                for tool in self.config.allowed_tools:
                    if tool.name == tool_name:
                        try:
                            result = tool.invoke(tool_call)
                            tool_msg = ToolMessage(
                                content=str(result),
                                tool_call_id=tool_call["id"],
                                name=tool_name
                            )
                            updated_state["messages"].append(tool_msg)
                        except Exception as e:
                            tool_msg = ToolMessage(
                                content=f"Error: {str(e)}",
                                tool_call_id=tool_call["id"]
                            )
                            updated_state["messages"].append(tool_msg)
                            console.print(f"[#FF6B6B]Tool Error:[/#FF6B6B] {str(e)}")

        return updated_state, next_agent

    def __call__(self, state: State) -> Command[Dict[str, Any]]:
        """Process state and determine next action"""
        try:
            messages = [
                SystemMessage(content=self.config.system_prompt)
            ] + state["messages"]

            response = model.bind_tools(
                self.config.allowed_tools
            ).invoke(messages)
            
            console.print(Panel(
                response.content,
                title=f"[#4CAF50]{self.config.agent_id}[/#4CAF50]",
                border_style="#4CAF50"
            ))

            updated_state, next_agent = self.process_tools(response, state)
            
            if next_agent:
                return Command(goto=next_agent, update=updated_state)
            
            if "FINAL ANSWER:" in response.content:
                return Command(goto="__end__", update=updated_state)
            
            return Command(goto="supervisor", update=updated_state)

        except Exception as e:
            console.print(f"[#FF6B6B]Agent Error:[/#FF6B6B] {str(e)}")
            error_state = state.copy()
            error_state["messages"] = state["messages"] + [
                AIMessage(content=f"Error: {str(e)}")
            ]
            return Command(goto="__end__", update=error_state)

class BaseTeam(Generic[T]):
    """Base class for all agent teams"""
    def __init__(
        self,
        team_id: str,
        agents: List[BaseAgent],
        supervisor_prompt: str,
        state_class: type[T] = State
    ):
        self.team_id = team_id
        self.agents = {agent.config.agent_id: agent for agent in agents}
        self.supervisor_prompt = supervisor_prompt
        self.state_class = state_class
        self.graph = self._build_graph()

    def _build_graph(self) -> StateGraph:
        """Build the agent interaction graph"""
        builder = StateGraph(self.state_class)
        for agent_id, agent in self.agents.items():
            builder.add_node(agent_id, agent)
        builder.add_node("supervisor", self._create_supervisor())
        builder.add_edge(START, "supervisor")
        return builder.compile()

    def _create_supervisor(self):
        """Create supervisor node"""
        def supervisor_node(state: T) -> Command[Dict[str, Any]]:
            system_msg = SystemMessage(content=self.supervisor_prompt)
            messages = [system_msg] + state["messages"]
            
            response = model.invoke(messages)
            console.print(Panel(
                response.content,
                title="[#E040FB]Supervisor Decision[/#E040FB]",
                border_style="#E040FB"
            ))
            
            next_agent = self._determine_next_agent(response)
            if next_agent == "__end__":
                return Command(goto="__end__", update={
                    "messages": state["messages"] + [response]
                })
                
            return Command(goto=next_agent, update={
                "messages": state["messages"] + [response]
            })
        
        return supervisor_node

    def _determine_next_agent(self, response: AIMessage) -> str:
        """Determine next agent - override in subclasses"""
        raise NotImplementedError("Subclasses must implement _determine_next_agent")

    def process(self, input_data: str) -> Dict[str, Any]:
        """Process input through the team"""
        console.print(Panel(
            input_data,
            title=f"[#00BCD4]{self.team_id} Input[/#00BCD4]",
            border_style="#00BCD4"
        ))
        state = self.state_class(
            messages=[HumanMessage(content=input_data)],
            context={}
        )
        return self.graph.invoke(state)

# ============= Calculator Implementation =============
class CalculatorTeam(BaseTeam[State]):
    """Team of calculator agents"""
    def _determine_next_agent(self, response: AIMessage) -> str:
        """Route based on operation needed"""
        content = response.content.lower()
        
        if "FINAL ANSWER:" in response.content:
            return "__end__"
            
        if any(phrase in content for phrase in [
            "need to multiply",
            "multiplication needed",
            "multiply these",
            "*", "times"
        ]):
            return "multiplication_expert"
            
        if any(phrase in content for phrase in [
            "need to add",
            "addition needed",
            "add these",
            "+"
        ]):
            return "addition_expert"
            
        return "__end__"

def create_calculator_team() -> CalculatorTeam:
    """Create calculator team instance"""
    add_agent = BaseAgent(
        config=AgentConfig(
            agent_id="addition_expert",
            role=AgentRole.WORKER,
            system_prompt=(
                "You are an addition expert. Your job is to:\n"
                "1. Look for numbers that need to be added\n"
                "2. Use the add tool to perform addition\n"
                "3. If multiplication is needed, transfer to multiplication_expert\n"
                "4. Show all work and intermediate steps\n"
                "5. When the final result is ready, say 'FINAL ANSWER: [result]'\n\n"
                "Remember: Only handle addition. Always transfer multiplication tasks."
            ),
            allowed_tools=[add, transfer_to_multiplication_expert],
            allowed_handoffs=["multiplication_expert"]
        )
    )

    mult_agent = BaseAgent(
        config=AgentConfig(
            agent_id="multiplication_expert",
            role=AgentRole.WORKER,
            system_prompt=(
                "You are a multiplication expert. Your job is to:\n"
                "1. Look for numbers that need to be multiplied\n"
                "2. Use the multiply tool to perform multiplication\n"
                "3. If addition is needed, transfer to addition_expert\n"
                "4. Show all work and intermediate steps\n"
                "5. When the final result is ready, say 'FINAL ANSWER: [result]'\n\n"
                "Remember: Only handle multiplication. Always transfer addition tasks."
            ),
            allowed_tools=[multiply, transfer_to_addition_expert],
            allowed_handoffs=["addition_expert"]
        )
    )

    return CalculatorTeam(
        team_id="calculator",
        agents=[add_agent, mult_agent],
        supervisor_prompt=(
            "You are a math supervisor. Your job is to:\n"
            "1. Analyze the expression and identify needed operations\n"
            "2. Route addition tasks to addition_expert\n"
            "3. Route multiplication tasks to multiplication_expert\n"
            "4. For complex expressions, follow order of operations\n"
            "5. Only identify the next operation - do NOT calculate\n\n"
            "Remember: Your job is to ROUTE tasks, not solve them."
        )
    )

def main():
    calculator = create_calculator_team()
    expression = "What is (17 + 23) * (14 + 16)?"
    
    try:
        result = calculator.process(expression)
        final_msg = result["messages"][-1].content
        if "FINAL ANSWER:" in final_msg:
            answer = final_msg.split("FINAL ANSWER:")[-1].strip()
            console.print(Panel(
                f"[#4CAF50]{answer}[/#4CAF50]",
                title="[#4CAF50]Final Answer[/#4CAF50]",
                border_style="#4CAF50"
            ))
        else:
            console.print("[#FF6B6B]No final answer found in response[/#FF6B6B]")
            
    except Exception as e:
        console.print(f"[#FF6B6B]Error:[/#FF6B6B] {str(e)}")

if __name__ == "__main__":
    main()

#### How to add multi-turn conversation in a multi-agent application


### State Management


#### How to use Pydantic model as state


#### How to define input/output schema for your graph


#### How to pass private state between nodes inside the graph


### Other


#### How to run graph asynchronously


#### How to visualize your graph


#### How to add runtime configuration to your graph


#### How to add node retries


#### How to force function calling agent to structure output


#### How to pass custom LangSmith run ID for graph runs


#### How to return state before hitting recursion limit


#### How to integrate LangGraph with AutoGen, CrewAI, and other frameworks


### Prebuilt ReAct Agent


#### How to create a ReAct agent


#### How to add memory to a ReAct agent


#### How to add a custom system prompt to a ReAct agent


#### How to add human-in-the-loop processes to a ReAct agent


#### How to create prebuilt ReAct agent from scratch


#### How to add semantic search for long-term memory to a ReAct agent


## LangGraph Platform


### Application Structure


#### How to set up app for deployment (requirements.txt)


#### How to set up app for deployment (pyproject.toml)


#### How to set up app for deployment (JavaScript)


#### How to add semantic search


#### How to customize Dockerfile


#### How to test locally


#### How to rebuild graph at runtime


#### How to use LangGraph Platform to deploy CrewAI, AutoGen, and other frameworks


### Deployment


#### How to deploy to LangGraph cloud


#### How to deploy to a self-hosted environment


#### How to interact with the deployment using RemoteGraph


### Authentication & Access Control


#### How to add custom authentication


#### How to update the security schema of your OpenAPI spec


### Assistants


#### How to configure agents


#### How to version assistants


### Threads


#### How to copy threads


#### How to check status of your threads


### Runs


#### How to run an agent in the background


#### How to run multiple agents in the same thread


#### How to create cron jobs


#### How to create stateless runs


### Streaming


#### How to stream values


#### How to stream updates


#### How to stream messages


#### How to stream events


#### How to stream in debug mode


#### How to stream multiple modes


### Human-in-the-loop


#### How to add a breakpoint


#### How to wait for user input


#### How to edit graph state


#### How to replay and branch from prior states


#### How to review tool calls


### Double-texting


#### How to use the interrupt option


#### How to use the rollback option


#### How to use the reject option


#### How to use the enqueue option


### Webhooks


#### How to integrate webhooks


### Cron Jobs


#### How to create cron jobs


### LangGraph Studio


#### How to connect to a LangGraph Cloud deployment


#### How to connect to a local dev server


#### How to connect to a local deployment (Docker)


#### How to test your graph in LangGraph Studio (MacOS only)


#### How to interact with threads in LangGraph Studio


#### How to add nodes as dataset examples in LangGraph Studio


### Troubleshooting


#### GRAPH_RECURSION_LIMIT


#### INVALID_CONCURRENT_GRAPH_UPDATE


#### INVALID_GRAPH_NODE_RETURN_VALUE


#### MULTIPLE_SUBGRAPHS


#### INVALID_CHAT_HISTORY

# Dev

# Ref

- https://langchain-ai.github.io/langgraph
- https://langchain-ai.github.io/langgraph/concepts/

- [Conceptual Guide](https://langchain-ai.github.io/langgraph/concepts/)
  - LangGraph
    - High Level
      - [Why LangGraph?](https://langchain-ai.github.io/langgraph/concepts/high_level/)
    - Concepts
      - [LangGraph Glossary](https://langchain-ai.github.io/langgraph/concepts/low_level/) 1️⃣
      - [Common Agentic Patterns](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/) 2️⃣
      - [Multi-Agent Systems](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#multi-agent-architectures) 1️⃣
      - Breakpoints
      - Human-in-the-Loop
      - Time Travel
      - Persistence
      - Memory
      - Streaming
      - FAQ
  - LangGraph Platform
    - High Level
      - Why LangGraph Platform?
      - Deployment Options
      - Plans
      - Template Applications
    - Components
      - LangGraph Server
      - LangGraph Studio
      - LangGraph CLI
      - Python/JS SDK
      - Remote Graph
    - LangGraph Server
      - Application Structure
      - Assistants
      - Web-hooks
      - Cron Jobs
      - Double Texting
      - Authentication & Access Control
    - Deployment Options
      - Self-Hosted Lite
      - Cloud SaaS
      - Bring Your Own Cloud
      - Self-Hosted Enterprise

- Tutorials
  - Quick Start
    - [🚀 LangGraph Quick Start](https://langchain-ai.github.io/langgraph/tutorials/introduction/) ✅
  - Chatbots
  - RAG
  - Agent Architectures
    - Multi-Agent Systems¶
      - [Network](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration/)
      - Supervisor
      - Hierarchical Teams
    - Planning Agents¶
      - Plan-and-Execute
      - Reasoning without Observation
      - LLMCompiler
    - Reflection & Critique¶
      - Basic Reflection
      - Reflexion
      - Tree of Thoughts
      - Language Agent Tree Search
      - Self-Discover Agent
  - Evaluation & Analysis
  - Experimental
  - LangGraph Platform

- How-to
  - LangGraph
    - Controllability
      - How to create branches for parallel execution
      - How to create map-reduce branches for parallel execution
      - How to control graph recursion limit
      - How to combine control flow and state updates with Command
    - Persistence
      - How to add thread-level persistence to your graph
      - How to add thread-level persistence to subgraphs
      - How to add cross-thread persistence to your graph
      - How to use Postgres checkpointer for persistence
      - How to use MongoDB checkpointer for persistence
      - How to create a custom checkpointer using Redis
    - Memory
      - How to manage conversation history
      - How to delete messages
      - How to add summary conversation memory
      - How to add long-term memory (cross-thread)
      - How to use semantic search for long-term memory
    - Human-in-the-loop
      - How to wait for user input
      - How to review tool calls
      - How to add static breakpoints
      - How to edit graph state
      - How to add dynamic breakpoints with NodeInterrupt
    - Time Travel
      - How to view and update past graph state
    - Streaming
    	- [How to stream](https://langchain-ai.github.io/langgraph/how-tos/streaming/#values) 1️⃣
			- [How to stream LLM tokens](https://langchain-ai.github.io/langgraph/how-tos/streaming-tokens/) 1️⃣
			- [How to stream LLM tokens from specific nodes](https://langchain-ai.github.io/langgraph/how-tos/streaming-specific-nodes/) 1️⃣
			- [How to stream data from within a tool](https://langchain-ai.github.io/langgraph/how-tos/streaming-events-from-within-tools/) 1️⃣
			- [How to stream from subgraphs](https://langchain-ai.github.io/langgraph/how-tos/streaming-subgraphs/) 1️⃣
			- How to disable streaming for models that don't support it
    - Tool calling
      - How to call tools using ToolNode
      - How to handle tool calling errors
      - How to pass runtime values to tools
      - How to pass config to tools
      - How to update graph state from tools
      - How to handle large numbers of tools
    - Subgraphs
      - [How to use subgraphs](https://langchain-ai.github.io/langgraph/how-tos/subgraph/) 2️⃣
      - [How to view and update state in subgraphs](https://langchain-ai.github.io/langgraph/how-tos/subgraphs-manage-state/) 🚧
      - How to transform inputs and outputs of a subgraph
    - Multi-agent
      - [How to implement handoffs between agents](https://langchain-ai.github.io/langgraph/how-tos/agent-handoffs/) 2️⃣
      - [How to build a multi-agent network](https://langchain-ai.github.io/langgraph/how-tos/multi-agent-network/) 2️⃣
      - How to add multi-turn conversation in a multi-agent application
    - State Management
      - How to use Pydantic model as state
      - How to define input/output schema for your graph
      - How to pass private state between nodes inside the graph
    - Other
      - How to run graph asynchronously
      - How to visualize your graph
      - How to add runtime configuration to your graph
      - How to add node retries
      - How to force function calling agent to structure output
      - How to pass custom LangSmith run ID for graph runs
      - How to return state before hitting recursion limit
      - How to integrate LangGraph with AutoGen, CrewAI, and other frameworks
    - Prebuilt ReAct Agent
      - How to create a ReAct agent
      - How to add memory to a ReAct agent
      - How to add a custom system prompt to a ReAct agent
      - How to add human-in-the-loop processes to a ReAct agent
      - How to create prebuilt ReAct agent from scratch
      - How to add semantic search for long-term memory to a ReAct agent

  - LangGraph Platform
    - Application Structure
      - How to set up app for deployment (requirements.txt)
      - How to set up app for deployment (pyproject.toml)
      - How to set up app for deployment (JavaScript)
      - How to add semantic search
      - How to customize Dockerfile
      - How to test locally
      - How to rebuild graph at runtime
      - How to use LangGraph Platform to deploy CrewAI, AutoGen, and other frameworks
    - Deployment
      - How to deploy to LangGraph cloud
      - How to deploy to a self-hosted environment
      - How to interact with the deployment using RemoteGraph
    - Authentication & Access Control
      - How to add custom authentication
      - How to update the security schema of your OpenAPI spec
    - Assistants
      - How to configure agents
      - How to version assistants
    - Threads
      - How to copy threads
      - How to check status of your threads
    - Runs
      - How to run an agent in the background
      - How to run multiple agents in the same thread
      - How to create cron jobs
      - How to create stateless runs
    - Streaming
      - How to stream values
      - How to stream updates
      - How to stream messages
      - How to stream events
      - How to stream in debug mode
      - How to stream multiple modes
    - Human-in-the-loop
      - How to add a breakpoint
      - How to wait for user input
      - How to edit graph state
      - How to replay and branch from prior states
      - How to review tool cal
    - Double-texting
      - How to use the interrupt option
      - How to use the rollback option
      - How to use the reject option
      - How to use the enqueue option
    - Webhooks
      - How to integrate webhooks
    - Cron Jobs
      - How to create cron jobs
    - LangGraph Studio
      - How to connect to a LangGraph Cloud deployment
      - How to connect to a local dev server
      - How to connect to a local deployment (Docker)
      - How to test your graph in LangGraph Studio (MacOS only)
      - How to interact with threads in LangGraph Studio
      - How to add nodes as dataset examples in LangGraph Studio
    - Troubleshooting
      - GRAPH_RECURSION_LIMIT
      - INVALID_CONCURRENT_GRAPH[](https://langchain-ai.github.io/langgraph/how-tos/streaming/#values)
      - INVALID_GRAPH_NODE_RETURN_VALUE
      - MULTIPLE_SUBGRAPHS
      - INVALID_CHAT_HISTORY
  
...